In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
#### Load the trained model, scaler pickle, onehot
model=load_model('model.h5')

### Load the scaler and one-hot encoder
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)


In [3]:
# Example imput data

input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':6000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [4]:

geo_encoded=label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))

geo_df

/Users/sushrinadhakal/.pyenv/versions/3.10.12/envs/tf-env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0

In [5]:
input_df=pd.DataFrame([input_data])

In [6]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

CreditScore Geography  Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France       1   40       3     6000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000

In [7]:
## concatenation of one hot encoded
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_df],axis=1)


In [8]:
input_df

CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          600       1   40       3     6000              2          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1            50000               1.0                0.0   

   Geography_Spain  
0              0.0

In [9]:
## Scaling the input data

input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -1.12240462,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [10]:
## Predict Churn

prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


array([[0.01612046]], dtype=float32)

In [11]:
prediction_probability=prediction[0][0]
if prediction_probability > 0.5:
    print("The customer is likely to churn with a probability of {:.2f}%".format(prediction_probability * 100))
else:
    print("The customer is unlikely to churn with a probability of {:.2f}%".format((1 - prediction_probability) * 100))

The customer is unlikely to churn with a probability of 98.39%
